In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 11.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=d4b8d28f9656f10b97d84a17f6859778d67625e5300bf7ad43b4b833d2306cbe
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from pyspark import SparkContext

In [3]:
sc = SparkContext()

In [4]:
%%writefile example.txt
first line
second line
third line
fourth line

Writing example.txt


In [6]:
textfile = sc.textFile('example.txt')

In [7]:
textfile.count()

4

In [11]:
textfile.first()

'first line'

In [8]:
filtered = textfile.filter(lambda x:'second' in x)

In [9]:
filtered.count()

1

In [10]:
filtered.collect()

['second line']

In [12]:
words = textfile.map(lambda line:line.split())

In [13]:
words.collect()

[['first', 'line'], ['second', 'line'], ['third', 'line'], ['fourth', 'line']]

In [14]:
words2 = textfile.flatMap(lambda line:line.split())

In [15]:
words2.collect()

['first', 'line', 'second', 'line', 'third', 'line', 'fourth', 'line']

In [16]:
%%writefile services.txt
#EventId    Timestamp    Customer   State    ServiceID    Amount
201       10/13/2017      100       NY       131          100.00
204       10/18/2017      700       TX       129          450.00
202       10/15/2017      203       CA       121          200.00
206       10/19/2017      202       CA       131          500.00
203       10/17/2017      101       NY       173          750.00
205       10/19/2017      202       TX       121          200.00

Writing services.txt


In [17]:
services = sc.textFile('services.txt')

In [18]:
services.take(2)

['#EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00']

In [19]:
services.map(lambda line:line.split()).take(3)

[['#EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00']]

In [23]:
clean = services.map(lambda line:line[1:].split() if line[0]=='#' else line.split())

In [31]:
pairs = clean.map(lambda lst:(lst[3],lst[5]))

In [32]:
keys = pairs.reduceByKey(lambda amt1,amt2 : float(amt1)+float(amt2))

In [33]:
keys.collect()

[('State', 'Amount'), ('NY', 850.0), ('TX', 650.0), ('CA', 700.0)]

In [35]:
step1 = clean.map(lambda lst: (lst[3],lst[-1]))
step2 = step1.reduceByKey(lambda amt1,amt2:float(amt1)+float(amt2))
step3 = step2.filter(lambda x: not x[0]=='State')
step4 = step3.sortBy(lambda stAmount: stAmount[1],ascending=False)
step4.collect()


[('NY', 850.0), ('CA', 700.0), ('TX', 650.0)]